In [26]:
import requests  # used to get url
import json  # used to read json files on the webpage
import pandas as pd  # used to integrate data

In [27]:
# There are 84 pages on the WIC website
page_count = 84

# Begin with page 1
page_number = 1

# start_url begins with page 1 as well
start_url = 'https://wicgrocerystores.web.health.state.mn.us/search?criteria=&size=10&page=' + str(page_number)

In [28]:
# Get response from start_url
resp = requests.get(start_url)

# Load the json file in start_url (only page 1 is there)
data = json.loads(resp.text)
print(data)

{'totalElements': 834, 'data': [{'id': '8245', 'name': '1st Quality Market', 'address': {'street': '2655 Nicollet Ave', 'city': 'MINNEAPOLIS', 'zipCode': 55408, 'county': 'HENNEPIN'}}, {'id': '1548', 'name': '52 MARKET  AND TRADING', 'address': {'street': '990 ARCADE ST', 'city': 'SAINT PAUL', 'zipCode': 55106, 'county': 'RAMSEY'}}, {'id': '8782', 'name': '75 Market and Deli', 'address': {'street': '1187 Minnehaha Ave E ', 'city': 'SAINT PAUL', 'zipCode': 55106, 'county': 'RAMSEY'}}, {'id': '8685', 'name': '7th Grocery', 'address': {'street': '43 7th St W', 'city': 'SAINT PAUL', 'zipCode': 55102, 'county': 'RAMSEY'}}, {'id': '8875', 'name': 'A A Market', 'address': {'street': '191 Western Ave N', 'city': 'SAINT PAUL', 'zipCode': 55102, 'county': 'RAMSEY'}}, {'id': '8552', 'name': 'Aaran Halal Market', 'address': {'street': '8904 Old Cedar Ave S', 'city': 'BLOOMINGTON', 'zipCode': 55425, 'county': 'HENNEPIN'}}, {'id': '8379', 'name': 'Africa International Market', 'address': {'street': 

In [29]:
# Normalize the json file into the pandas data frame called "df".
df = pd.io.json.json_normalize(data['data'])
df

,id,name,address.street,address.city,address.zipCode,address.county
0,8245,1st Quality Market,2655 Nicollet Ave,MINNEAPOLIS,55408,HENNEPIN
1,1548,52 MARKET AND TRADING,990 ARCADE ST,SAINT PAUL,55106,RAMSEY
2,8782,75 Market and Deli,1187 Minnehaha Ave E,SAINT PAUL,55106,RAMSEY
3,8685,7th Grocery,43 7th St W,SAINT PAUL,55102,RAMSEY
4,8875,A A Market,191 Western Ave N,SAINT PAUL,55102,RAMSEY
5,8552,Aaran Halal Market,8904 Old Cedar Ave S,BLOOMINGTON,55425,HENNEPIN
6,8379,Africa International Market,7617 Welcome Ave N,BROOKLYN PARK,55443,HENNEPIN
7,8783,African Halal & Deli,405 E Lake St,MINNEAPOLIS,55408,HENNEPIN
8,8624,African Plaza,555 Snelling Ave N,SAINT PAUL,55104,RAMSEY
9,341,AFRIK GROCERY,613 CEDAR AVE S,MINNEAPOLIS,55454,HENNEPIN


In [30]:
# Store 840 grocery stores into a pandas data frame called "df2".
df2 = pd.DataFrame()

# Here I create a for loop to get through all pages
for page_number in range(page_count):  
    page_number = page_number + 1
    start_url = 'https://wicgrocerystores.web.health.state.mn.us/search?criteria=&size=10&page=' + str(page_number)
    resp_next = requests.get(start_url)
    data = json.loads(resp_next.text)
    df = pd.io.json.json_normalize(data['data'])
    df2 = df2.append(df)

In [31]:
# Check if the loop runs to the end
print(page_number)
print(start_url)

84
https://wicgrocerystores.web.health.state.mn.us/search?criteria=&size=10&page=84


In [32]:
# Get the total number of store records
len(df2)

834

In [33]:
# Get the total number of store records
len(df2)

834

In [34]:
import googlemaps

# Replace "MY_API_KEY" with your API key
my_api_key = "MY_API_KEY"  

In [35]:
# Get my Google Maps Platform API key
gmaps_key = googlemaps.Client(key = my_api_key) 

In [36]:
# A Text Search request is an HTTP URL of the following form:
gmaps_text_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"

In [37]:
df_loc = df2.reset_index(drop=True) 

In [38]:
record_count = len(df_loc)

In [39]:
for index in range(record_count):
    if '#' in df_loc.at[index,'name'] != None:
        df_loc.loc[index,'name'] = df_loc.at[index,'name'].split('#')[0]

In [44]:
lat = []
long = []

In [45]:
for index in range(record_count): # which is supposed to be record_count !!!! 
    query = df_loc.at[index,'address.street'] + ', ' + df_loc.at[index,'address.city']+ ', ' + str(df_loc.at[index,'address.zipCode'])+ ', ' + df_loc.at[index,'name']
    gmaps_search_url = gmaps_text_url + 'query=' + query + '&key=' + my_api_key
    gmaps_search_resp = requests.get(gmaps_search_url)
    gmaps_place_data = gmaps_search_resp.json()
    
    if gmaps_place_data['status'] == 'ZERO_RESULTS':
        lat.append('NaN')
        long.append('NaN')
    else: 
        try:
            lat.append(gmaps_place_data['results'][0]['geometry']['location']['lat'])    
            long.append(gmaps_place_data['results'][0]['geometry']['location']['lng'])
        except:
            lat.append('NaN')
            long.append('NaN')

In [47]:
print(len(lat))
print(len(long))

834
834


In [48]:
new_cols = ['lat', 'long']

In [49]:
coordinates = {'lat': lat, 'long': long}
df_coor = pd.DataFrame(coordinates, columns=new_cols)

In [50]:
df9 = pd.concat([df_loc, df_coor], axis=1)

In [51]:
df9.to_csv("locations.csv")